In [5]:
"""
 - [ ] Activation functions
 - [ ] Batch Normalization. 
 - [ ] Residual connections 
 - [ ] the Adam optimizer
 - [ ] Backpropagation
"""
# [Paper]()

'\n - [ ] Batch Normalization. \n - [ ] Residual connections \n - [ ] the Adam optimizer\n'